<a href="https://colab.research.google.com/github/Pebiebi/Machine-Learning-with-Python-Projects---Neural-Network-SMS-Text-Classifier/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 641.8/641.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 55.9 MB/s eta 0:00:00
2.19.0-dev20250121


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-01-22 20:37:18--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  1.12MB/s    in 0.3s    

2025-01-22 20:37:18 (1.12 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2025-01-22 20:37:18--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [5]:
# Load data
train_data = pd.read_csv("train-data.tsv", sep="\t", header=None, names=["label", "message"])
valid_data = pd.read_csv("valid-data.tsv", sep="\t", header=None, names=["label", "message"])

In [6]:
# Prepare the data
train_messages = train_data["message"].values
train_labels = (train_data["label"] == "spam").astype(int).values

valid_messages = valid_data["message"].values
valid_labels = (valid_data["label"] == "spam").astype(int).values

In [7]:
# Tokenization and padding
vocab_size = 10000
max_length = 50
embedding_dim = 16
oov_token = "<OOV>"

In [8]:
# Tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_messages)

train_sequences = tokenizer.texts_to_sequences(train_messages)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')

valid_sequences = tokenizer.texts_to_sequences(valid_messages)
valid_padded = pad_sequences(valid_sequences, maxlen=max_length, padding='post', truncating='post')

In [9]:
# Model definition
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [10]:
# Train the model
epochs = 10
batch_size = 32

model.fit(train_padded, train_labels, epochs=epochs, validation_data=(valid_padded, valid_labels), batch_size=batch_size)

AttributeError: module 'ml_dtypes' has no attribute 'float4_e2m1fn'
Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 16s 74ms/step - accuracy: 0.8699 - loss: 0.4517 - val_accuracy: 0.8657 - val_loss: 0.3275
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 73ms/step - accuracy: 0.9293 - loss: 0.1942 - val_accuracy: 0.8793 - val_loss: 0.2096
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 11s 76ms/step - accuracy: 0.9041 - loss: 0.1975 - val_accuracy: 0.9088 - val_loss: 0.1924
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 9s 72ms/step - accuracy: 0.9487 - loss: 0.1447 - val_accuracy: 0.9569 - val_loss: 0.1551
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 9s 62ms/step - accuracy: 0.9687 - loss: 0.1362 - val_accuracy: 0.9641 - val_loss: 0.1470
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 60ms/step - accuracy: 0.9742 - loss: 0.1186 - val_accuracy: 0.9670 - val_loss: 0.1472
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 11s 66ms/step - accuracy: 0.9809 - loss: 0.0953 - val_accuracy: 0.9662 - val_loss: 0.1434
Epoch 8/10
131/131 ━━━━━

In [11]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    # Convert text to sequences and pad
    sequence = tokenizer.texts_to_sequences([pred_text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post', truncating='post')

    # Get prediction
    prediction_prob = model.predict(padded_sequence)[0][0]
    label = "spam" if prediction_prob > 0.5 else "ham"

    return [float(prediction_prob), label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 488ms/step
[0.0062896073795855045, 'ham']


In [12]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
You passed the challenge. Great job!
